# Read Filtering for Assembly Workflow
This noteboook installs all docker containers; creates all directories; and runs all programs neccessary to implement a read filtering pipeline for use with metagenomics assembler software. For assembly minimal trimming is performed and the primary goal is to remove aapter sequences. The workflow first enables downloading and generating of all required docker containers, inputs, and drectories. Quality assesments are then executed using FastQC. Datasets are trimmed of adapter TruSeq sequences using Trimmomatic. FastQC dataset analyses are re-calculated over the trimmed datasets. Read 1 and Read 2 sequences are interleaven using Khmer. The example datasets provided as sample imputs are a subset of the Shakya M. et al.(doi 10.1111/1462-2920.12086) dataset which is described in the dataset-characteristics.ipynb notebook.

### Download and Install Required Docker Containers

In [8]:
# Install docker containers to run pipeline
!docker pull biocontainers/fastqc
!docker pull quay.io/biocontainers/trimmomatic:0.36--4
!docker pull quay.io/biocontainers/khmer:2.1--py35_0
!docker pull quay.io/biocontainers/pandaseq:2.11--1

Using default tag: latest
latest: Pulling from biocontainers/fastqc
Digest: sha256:7839537b143498ccc32214c4b02a235f7a10b0794064647323de0d5d5957f42b
Status: Image is up to date for biocontainers/fastqc:latest
0.36--4: Pulling from biocontainers/trimmomatic

95caeb02: Already exists 
a756c345: Already exists 
60de4b27: Already exists 
c29a56fb: Already exists 
bb6634fc: Already exists 
10677cff: Already exists 
95caeb02: Already exists 
b3b2fa0d: Already exists 
Digest: sha256:f162a5c4f2644a00e186a6e105e798d4bcdecddb653474ea7e42c915dc6a416b
Status: Image is up to date for quay.io/biocontainers/trimmomatic:0.36--4
2.1--py35_0: Pulling from biocontainers/khmer

95caeb02: Already exists 
c00e8b61: Already exists 
de50789a: Already exists 
8b9f3d2a: Already exists 
99a2256f: Already exists 
336f2e44: Already exists 
95caeb02: Already exists 
bb32200b: Already exists 
1b1b4b4f: Already exists Digest: sha256:f82b5f6cc5d57ca9e1b4854f41ea088debaaf2d0b6043a7dc874466433ff35d1
Status: Image is up t

### Set up directories and download input datasets

In [22]:
# Make a working directory (/home/username/data) and download input datasets
!mkdir /home/ubuntu/data
!cd /home/ubuntu/data
!pip install osfclient
!osf -p dm938 fetch osfstorage/data/SRR606249_subset10_1.fq.gz
!osf -p dm938 fetch osfstorage/data/SRR606249_subset10_2.fq.gz


### Run FastQC quality assesment tool over inputs

In [5]:
!pip install beautifulsoup4

    100% |████████████████████████████████| 92kB 981kB/s ta 0:00:01


In [6]:
import base64
import argparse
from bs4 import BeautifulSoup

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="python module to pull images from the HTML output file")
    parser.add_argument("html_file", help="HTML file to get images from ")
    args = parser.parse_args()
    in_file = args.html_file 
    
    #"SRR606249_subset10_1.fq.gz FastQC Report.html"
    soup = BeautifulSoup(open(in_file),"html5lib")
    graph_alt_names = ["Per base quality graph", "Per sequence quality graph", "Per base sequence content", "Per sequence GC content graph", "N content graph", 
                       "Sequence Length distribution", "Duplication level graph",  "Adapter graph", "Kmer graph" ]
  
    for image in soup.find_all('img', alt=True):
        if (image['alt'] in graph_alt_names):
            base64_image_str = image['src']
            base64_image_str = base64_image_str[base64_image_str.find(",")+1:]
            base64_image_str_bytes = bytes(base64_image_str, encoding="UTF-8")
            image_64_decode = base64.decodestring(base64_image_str_bytes)
            with open(image["alt"], "wb")as f:
                f.write(image_64_decode)
            


usage: ipykernel_launcher.py [-h] html_file
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/home/oana/py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [52]:
%%HTML
<iframe src="/home/oana/data2/SRR606249_subset10_1_fastqc.html" width=300 height=800></iframe>

In [24]:
#Run fastqc
!docker run -v /home/ubuntu/data:/data -it biocontainers/fastqc fastqc /data/SRR606249_subset10_1.fq.gz -o /data/
!docker run -v /home/ubuntu/data:/data -it biocontainers/fastqc fastqc /data/SRR606249_subset10_2.fq.gz -o /data/

Started analysis of SRR606249_subset10_1.fq.gz
Approx 5% complete for SRR606249_subset10_1.fq.gz
Approx 10% complete for SRR606249_subset10_1.fq.gz
Approx 15% complete for SRR606249_subset10_1.fq.gz
Approx 20% complete for SRR606249_subset10_1.fq.gz
Approx 25% complete for SRR606249_subset10_1.fq.gz
Approx 30% complete for SRR606249_subset10_1.fq.gz
Approx 35% complete for SRR606249_subset10_1.fq.gz
Approx 40% complete for SRR606249_subset10_1.fq.gz
Approx 45% complete for SRR606249_subset10_1.fq.gz
Approx 50% complete for SRR606249_subset10_1.fq.gz
Approx 55% complete for SRR606249_subset10_1.fq.gz
Approx 60% complete for SRR606249_subset10_1.fq.gz
Approx 65% complete for SRR606249_subset10_1.fq.gz
Approx 70% complete for SRR606249_subset10_1.fq.gz
Approx 75% complete for SRR606249_subset10_1.fq.gz
Approx 80% complete for SRR606249_subset10_1.fq.gz
Approx 85% complete for SRR606249_subset10_1.fq.gz
Approx 90% complete for SRR606249_subset10_1.fq.gz
Approx 95% complete for SRR606249_su

from IPython.display import display
display(HTML(filename='/home/oana/data2/SRR606249_subset10_1_fastqc.html'))


### Run Trimmomaitc
Download TruSeq adapter sequences. Trim the adapters from the datasets, and trim the datasets to a quality score of 2.

In [7]:
#Download TruSeq adapter sequences
!curl -O http://dib-training.ucdavis.edu.s3.amazonaws.com/mRNAseq-semi-2015-03-04/TruSeq2-PE.fa
!mv TruSeq2-PE.fa /home/oana/data2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   539  100   539    0     0      0      0 --:--:-- --:--:-- --:--:--     0 0     0   1743      0 --:--:-- --:--:-- --:--:--  1744


In [9]:
%%bash
for filename in *_1*.fq.gz
do
    # first, make the base by removing .fq.gz using the unix program basename
    base=$(basename $filename .fq.gz)
    echo $base

    # now, construct the base2 filename by replacing _1 with _2
    base2=${base/_1/_2}
    echo $base2

    docker run -v /home/oana/data2:/data -i quay.io/biocontainers/trimmomatic:0.36--4 trimmomatic PE /data/${base}.fq.gz \
        /data/${base2}.fq.gz \
        /data/${base}.trim.fq.gz /data/${base}_se \
        /data/${base2}.trim.fq.gz /data/${base2}_se \
        ILLUMINACLIP:TruSeq2-PE.fa:2:40:15 \
        LEADING:2 TRAILING:2 \
        SLIDINGWINDOW:4:2 \
        MINLEN:25
done


SRR606249_subset10_1
SRR606249_subset10_2


/usr/local/bin/trimmomatic: line 6: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
TrimmomaticPE: Started with arguments:
 /data/SRR606249_subset10_1.fq.gz /data/SRR606249_subset10_2.fq.gz /data/SRR606249_subset10_1.trim.fq.gz /data/SRR606249_subset10_1_se /data/SRR606249_subset10_2.trim.fq.gz /data/SRR606249_subset10_2_se ILLUMINACLIP:TruSeq2-PE.fa:2:40:15 LEADING:2 TRAILING:2 SLIDINGWINDOW:4:2 MINLEN:25
Multiple cores found: Using 2 threads
java.io.FileNotFoundException: /TruSeq2-PE.fa (No such file or directory)
	at java.io.FileInputStream.open0(Native Method)
	at java.io.FileInputStream.open(FileInputStream.java:195)
	at java.io.FileInputStream.<init>(FileInputStream.java:138)
	at org.usadellab.trimmomatic.fasta.FastaParser.parse(FastaParser.java:54)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.loadSequences(IlluminaClippingTrimmer.java:110)
	at org.usadellab.trimmomatic.trim.IlluminaClippingTrimmer.makeIlluminaClippingTrimme

### Run FastQC quality assesment tool over trimmed data

In [ ]:
#Re-asses using FastQC
docker run -v /home/oana/data2:/data -it biocontainers/fastqc fastqc /data/SRR606249_subset10_1.trim.fq.gz -o /data/
docker run -v /home/oana/data2:/data -it biocontainers/fastqc fastqc /data/SRR606249_subset10_2.trim.fq.gz -o /data/    

### Interleave paired-end reads using Khmer 
The output file name includes 'trim2' indicating the reads were trimmed at a quality score of 2. 

In [ ]:
%%bash
for filename in *_1.trim.fq.gz
do
    # first, make the base by removing _1.trim.fq.gz with basename
    base=$(basename $filename _1.trim.fq.gz)
    echo $base

    # construct the output filename
    output=${base}.pe.trim2.fq.gz

    docker run -v /home/oana/data2:/data -it quay.io/biocontainers/khmer:2.1--py35_0 interleave-reads.py \
        /data/${base}_1.trim.fq.gz /data/${base}_2.trim.fq.gz --no-reformat -o /data/$output --gzip

done